In [1]:
!pip install praw pandas numpy scikit-learn matplotlib scipy PyDrive
!pip install gensim nltk 

In [2]:
import os
import praw
import pickle
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\h

[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank 

[nltk_data]    |   Package perluniprops is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package vader_lexicon to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package vader_lexicon is already up-to-date!
[nltk_data]    | Downloading package porter_test to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package porter_test is already up-to-date!
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wmt15_eval is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | 
[nltk_data

In [8]:
sym1 = re.compile('[/(){}\[\]\|@,;]')
sym2 = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [6]:
#utitlity functions
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def to_string(t):
    return str(t)

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = sym1.sub(' ',text)
    text = sym2.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [7]:
#preparing training data
flair_type = ["Reddiquette","Policy","Politics","AskIndia","Non-Political"]
reddit = praw.Reddit(client_id='#######', 
                     client_secret='##############', 
                     user_agent='#########', 
                     username='##########',  
                     password='#########')

subreddit = reddit.subreddit('india')
records_dict = {"flair":[], "title":[], "id":[], "url":[], "created": [], "body":[], "comments":[]}

for flair in flair_type:
  
  get_subreddits = subreddit.search(flair, limit=40)
  
  for x in get_subreddits:
    
    records_dict["flair"].append(flair)
    records_dict["title"].append(x.title)
    records_dict["id"].append(x.id)
    records_dict["url"].append(x.url)
    records_dict["created"].append(x.created)
    records_dict["body"].append(x.selftext)
    
    x.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in x.comments:
      comment = comment + ' ' + top_level_comment.body
    records_dict["comments"].append(comment)
    
record_data = pd.DataFrame(records_dict)
_timestamp = record_data["created"].apply(get_date)
record_data = record_data.assign(timestamp = _timestamp)
del record_data['created']
record_data.to_csv(r'C:\Users\harsh\RA_redditflair\csv_data\redditdata.csv',index = False)

In [10]:
#reading data from csvfile
topics_data = pd.read_csv(r'C:\Users\harsh\RA_redditflair\csv_data\redditdata.csv')


In [11]:
#text preprocessing
record_data['title'] = record_data['title'].apply(to_string)
record_data['body'] = record_data['body'].apply(to_string)
record_data['comments'] = record_data['comments'].apply(to_string)

record_data['title'] = record_data['title'].apply(clean_text)
record_data['body'] = record_data['body'].apply(clean_text)
record_data['comments'] = record_data['comments'].apply(clean_text)

basic_feature = record_data["title"] + record_data["comments"] + record_data["url"]
record_data = record_data.assign(basic_feature = basic_feature)

In [12]:
#splitting train and test data
X = record_data.basic_feature
y= record_data.flair
X_train , X_test , y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [13]:
#Linear Support Vecor Machine Model- Highest precision
from sklearn.linear_model import SGDClassifier
sgd = Pipeline([('c_vect',CountVectorizer()),#representing data in vector form(frequency)
            ('tfidf', TfidfTransformer()),#transform a count matrix to a normalized tf-idf form
            ('clf', SGDClassifier(loss = 'hinge', penalty = 'l2', alpha=1e-3, random_state = 42, max_iter =5 , tol = None))])
SGD = sgd.fit(X_train, y_train)
model = SGD
y_pred = sgd.predict(X_test)
    
print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names = flair_type))                

accuracy 0.6166666666666667
               precision    recall  f1-score   support

  Reddiquette       0.60      0.43      0.50        14
       Policy       0.75      0.82      0.78        11
     Politics       0.64      0.64      0.64        14
     AskIndia       0.50      0.45      0.48        11
Non-Political       0.57      0.80      0.67        10

    micro avg       0.62      0.62      0.62        60
    macro avg       0.61      0.63      0.61        60
 weighted avg       0.61      0.62      0.61        60



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [14]:
#saving model
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))